In [1]:
%pip install transformers torch datasets

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/471.6 kB ? eta -:--:--
   --- ----------------------------------- 41.0/471.6 kB 991.0 kB/s eta 0:00:01
   ----------------- ---------------------- 204.8/471.6 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------  471.0/471.6 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 471.6/471.6 kB 3.3 MB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   - -------------------------------------- 0.6/25.1 MB 13.3 MB/s eta 0:00:02
   -- ------------------------------------- 1.8/25.1 MB 19.0 MB/s eta 0:00:02
   ---- ----------------------------------- 3.0/25.1 MB 21

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orange3 3.36.2 requires xgboost>=1.7.4, which is not installed.
orange3 3.36.2 requires httpx>=0.21.0, but you have httpx 0.13.3 which is incompatible.


In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch

# Load pre-trained GPT-2 tokenizer and model
model_name = "gpt2"  # Use a GPT-2 variant, possibly fine-tune a Vietnamese model later
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add special tokens if needed (Vietnamese uses characters requiring special tokens)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Load the dataset (assuming you have a .txt file with couplets, each line a couplet)
def load_couplets(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return lines

# Create the dataset from the loaded text
couplets = load_couplets('train.txt')
dataset = Dataset.from_dict({"text": couplets})

# Tokenize the dataset for the model
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=50)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./results",              # Output directory
    overwrite_output_dir=True,           # Overwrite the output directory
    num_train_epochs=3,                  # Number of epochs
    per_device_train_batch_size=4,       # Batch size for training
    save_steps=10_000,                   # Save model every 10,000 steps
    save_total_limit=2,                  # Only keep the 2 most recent models
    logging_dir="./logs",                # Logging directory
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')


KeyboardInterrupt: 

ImportError: numpy.core.multiarray failed to import